In [1]:
import numpy as np
import pandas as pd
import pyodbc 
import mysql.connector
from datetime import datetime

In [2]:
date = '2021-09-01'
ASM = 'Ánh'

selected_date = "'{}%'".format(date)
if(ASM == ''):  
    selected_ASM = "Á"
elif(ASM == ''):
    selected_ASM = "T"
else:
    selected_ASM = "H"

In [92]:
final_df = pd.DataFrame(columns = [])

In [93]:
date = '2021-09-01'

for ASM in ["", "", ""]:
    selected_date = "'{}%'".format(date)
    if(ASM == ''):  
        selected_ASM = ""
    elif(ASM == ''):
        selected_ASM = ""
    else:
        selected_ASM = ""

    # Fetch all data with type=1 for subtraction
    connection = mysql.connector.connect(host='',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT b.id, b.depotId, b.createdDateTime, saleId, discount, money, usedPointsMoney, d.ASM, d.Store
    FROM . as b

    LEFT JOIN . as d ON b.depotId=d.DepotId
    WHERE b.Type = 1 AND b.Mode = 2 AND d.ASM LIKE '{}%' AND b.createdDateTime LIKE {}
    '''.format(selected_ASM, selected_date)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    data_df = pd.DataFrame(data, columns=cursor.column_names)

    # Save for analysis
    df = data_df

    df["mone"] = df["money"].apply(lambda x: float(x))
    df["usedPointsMoney"] = df["usedPointsMoney"].apply(lambda x: float(x))

    # Doanh Thu = DTLD - Tieu Diem
    df["Doanh Thu"] = df["money"] - df["usedPointsMoney"]

    sum_money = df.groupby("saleId")['Doanh Thu'].sum()

    # generate a string form to query order line items in database
    arr = ""

    arr = arr + "("
    for i in np.unique(df.id.values):
        arr = arr + str(i)
        arr = arr + ", "

    arr = arr[:-2]
    arr = arr + ")"

    # Query order details for each id
    connection = mysql.connector.connect(host='',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT * FROM .
    WHERE billId IN {};
    '''.format(arr)

    print(mySql_Create_Table_Query)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    product_df = pd.DataFrame(data, columns=cursor.column_names)


    product_df["code3"] = product_df["code"].apply(lambda x: x[:3])

    def func(x):
        if(x == "KID"):
            return "kid"
        elif(x == "APM" or x == "APN"):
            return "polo"
        else:
            return "sale_60%"

    product_df["marked"] = product_df.code3.apply(lambda x: func(x))

    product_df["price"] = product_df["price"].apply(lambda x: float(x))
    product_df["discount"] = product_df["discount"].apply(lambda x: float(x))
    product_df["money"] = product_df["money"].apply(lambda x: float(x))

    saleid_df = df[["saleId", "id"]]
    saleid_df.rename(columns={"id": "billId"}, inplace=True)

    total_df = product_df.merge(saleid_df, how="left", on="billId")

    marked_df = total_df.groupby(["saleId", "marked"])['money'].sum()
    marked_df2 = total_df.groupby(["saleId", "marked"])['money'].count()

    marked_df = pd.concat([marked_df, marked_df2], axis=1)
    marked_df.columns = ["money", "SL"]

    category_df = marked_df.unstack().reset_index()

    total_sale_df = pd.DataFrame(sum_money).reset_index()
    result = total_sale_df.merge(category_df, how="left", on="saleId")

    # Save doanh thu tra
    dtt_df = result

    # Doanh Thu Len Don - Doanh Thu Tra -> OKELA
    # Fetch all data with type=1 for subtraction

    connection = mysql.connector.connect(host='m',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT b.id, b.depotId, b.createdDateTime, saleId, discount, money, usedPointsMoney, d.ASM, d.Store
    FROM . as b

    LEFT JOIN . as d ON b.depotId=d.DepotId
    WHERE b.Type = 2 AND b.Mode = 2 AND d. LIKE '{}%' AND b.createdDateTime LIKE {}
    '''.format(selected_ASM, selected_date)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    data_df = pd.DataFrame(data, columns=cursor.column_names)

    # Save for analysis
    df = data_df

    df["money"] = df["money"].apply(lambda x: float(x))
    df["usedPointsMoney"] = df["usedPointsMoney"].apply(lambda x: float(x))

    # Doanh Thu = DTLD - Tieu Diem
    df["Doanh Thu"] = df["money"] - df["usedPointsMoney"]

    sum_money = df.groupby("saleId")['Doanh Thu'].sum()

    # generate a string form to query order line items in database
    arr = ""

    arr = arr + "("
    for i in np.unique(df.id.values):
        arr = arr + str(i)
        arr = arr + ", "

    arr = arr[:-2]
    arr = arr + ")"

    # Query order details for each id
    connection = mysql.connector.connect(host='',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT * FROM .
    WHERE billId IN {};
    '''.format(arr)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    product_df = pd.DataFrame(data, columns=cursor.column_names)


    product_df["code3"] = product_df["code"].apply(lambda x: x[:3])

    def func(x):
        if(x == "KID"):
            return "kid"
        elif(x == "APM" or x == "APN"):
            return "polo"
        else:
            return "sale_60%"

    product_df["marked"] = product_df.code3.apply(lambda x: func(x))

    product_df["price"] = product_df["price"].apply(lambda x: float(x))
    product_df["discount"] = product_df["discount"].apply(lambda x: float(x))
    product_df["money"] = product_df["money"].apply(lambda x: float(x))

    saleid_df = df[["saleId", "id"]]
    saleid_df.rename(columns={"id": "billId"}, inplace=True)

    total_df = product_df.merge(saleid_df, how="left", on="billId")

    marked_df = total_df.groupby(["saleId", "marked"])['money'].sum()
    marked_df2 = total_df.groupby(["saleId", "marked"])['money'].count()

    marked_df = pd.concat([marked_df, marked_df2], axis=1)
    marked_df.columns = ["money", "SL"]

    category_df = marked_df.unstack().reset_index()

    total_sale_df = pd.DataFrame(sum_money).reset_index()
    result = total_sale_df.merge(category_df, how="left", on="saleId")

    # result: dtld, dtt_df: dtt
    result.fillna(0.00, inplace=True)
    dtt_df.fillna(0.00, inplace=True)

    index_df = result[["saleId"]]

    dtt = index_df.merge(dtt_df, how="left", on="saleId").fillna(0.000)
    cols = result.columns.difference(['saleId'])
    result[cols] = result[cols].sub(dtt[cols])

    result["%polo"] = round(result[('money', 'polo')] / result["Doanh Thu"] * 100, 2)
    result["%sale_60%"] = round(result[('money', 'sale_60%')] / result["Doanh Thu"] * 100, 2)
    result["%kid"] = round(result[('money', 'kid')] / result["Doanh Thu"] * 100, 2)

    result["Date"] = date
    result["ASM"] = ASM

    final_df = final_df.append(result)
    
    print("--------------NEXT---INTERATION--------------")


    SELECT * FROM YODY_ORDER.nhanh_prod_table
    WHERE billId IN (175112807, 175113240, 175113407, 175113611, 175113853, 175113865, 175114013, 175114091, 175114237, 175114755, 175114859, 175114930, 175115075, 175115247, 175115423, 175115576, 175115594, 175115665, 175115978, 175116197, 175116527, 175116803, 175116815, 175116941, 175116952, 175116959, 175116968, 175117210, 175117495, 175117696, 175118442, 175118446, 175118598, 175118623, 175118642, 175118741, 175118826, 175118870, 175118952, 175119041, 175119116, 175119173, 175119253, 175119270, 175119278, 175119309, 175119402, 175119407, 175119653, 175119727, 175119765, 175119962, 175120186, 175120255, 175120336, 175120364, 175120396, 175120459, 175120481, 175120490, 175120602, 175120725, 175120928, 175120936, 175121186, 175121194, 175121414, 175122263, 175122398, 175122418, 175122448, 175122451, 175122494, 175122547, 175122605, 175122623, 175122667, 175122688, 175122703, 175122777, 175122795, 175122851, 175122863, 175122935, 17512296

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


--------------NEXT---INTERATION--------------

    SELECT * FROM YODY_ORDER.nhanh_prod_table
    WHERE billId IN (175111980, 175112876, 175112929, 175113302, 175113587, 175113598, 175113752, 175113885, 175113887, 175113930, 175114032, 175114182, 175114209, 175114373, 175114412, 175114461, 175114604, 175114664, 175114688, 175114695, 175114804, 175114944, 175114952, 175115098, 175115438, 175115486, 175115702, 175115972, 175116311, 175116416, 175116565, 175116926, 175116998, 175117090, 175117111, 175117120, 175117200, 175117421, 175117525, 175117731, 175118172, 175118668, 175118914, 175119073, 175119282, 175119340, 175119371, 175119530, 175119582, 175119607, 175119623, 175119635, 175119784, 175119820, 175119837, 175119883, 175119933, 175120428, 175120441, 175120493, 175120575, 175121061, 175121390, 175121500, 175121566, 175121580, 175121621, 175121675, 175122253, 175122333, 175122385, 175122601, 175122684, 175122729, 175122893, 175122898, 175122941, 175123021, 175123069, 175123133, 175123

--------------NEXT---INTERATION--------------

    SELECT * FROM YODY_ORDER.nhanh_prod_table
    WHERE billId IN (175114172, 175114601, 175115443, 175115724, 175116705, 175117067, 175117275, 175117688, 175118666, 175118916, 175119957, 175120126, 175121014, 175122551, 175122704, 175123464, 175123483, 175123652, 175124066, 175124711, 175125361, 175126249, 175127159, 175127266, 175127414, 175127689, 175129229, 175129423, 175129571, 175129706, 175129851, 175130057, 175130332, 175130356, 175131168, 175131706, 175131913, 175131947, 175131961, 175132366, 175132980, 175136378, 175136713, 175136914, 175139629, 175141353, 175141617, 175142160, 175142760, 175142928, 175143793, 175144438, 175144917, 175144996, 175146466, 175146874, 175149991, 175150275, 175151540, 175151570, 175152789, 175153338, 175153596, 175155772, 175156371, 175156461, 175156507, 175156577, 175157528, 175157647, 175157740, 175157993, 175158184, 175158493, 175158704, 175159208, 175159570, 175159748, 175159915, 175159930, 175160

In [69]:
final_df.to_pickle("1-2.pkl")

In [110]:
dff = pd.read_pickle("1-2.pkl")

In [111]:
dff["check_sum"] = dff[('money', 'kid')] + dff[('money', 'polo')] + dff[('money', 'sale_60%')]

In [66]:
final_df.to_pickle("1-9.pkl")

In [53]:
final_df = pd.read_pickle("1-9.pkl")

In [87]:
dsnv = pd.read_excel("dsnv.xlsx")

In [112]:
dsnv = pd.read_excel("dsnv.xlsx")
dsnv["Cửa hàng"] = dsnv["Cửa hàng"].str.split('\n').str[-1]

dff.dropna(subset=["saleId"], inplace=True)
dff.replace({'saleId': {"": 0}}, inplace=True)
dff.saleId = dff.saleId.astype(str).astype(np.int64)

dff = pd.merge(dsnv, dff, how="right", right_on="saleId", left_on="ID")

result_rearranged = dff[['ID', 'Tên đăng nhập', 'Tên đầy đủ', 'Nhóm quyền', 'Điện thoại',
       'Email', 'Cửa hàng', 'Date', 'Doanh Thu',  ('money', 'kid'),
            ('money', 'polo'),  ('money', 'sale_60%'),          ('SL', 'kid'),
               ('SL', 'polo'),     ('SL', 'sale_60%'),                '%polo',
                  '%sale_60%',                 '%kid', 'ASM', "check_sum"]]

result_rearranged.to_excel("final.xlsx")
final = pd.read_excel("final.xlsx", header=None)
final.drop(columns=[0], inplace=True)

C:\Users\Admin\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [113]:
# ACCES GOOGLE SHEET
import gspread
from gspread_dataframe import set_with_dataframe

sheet_id = '12a-yoM2IR3DBa0YeudUiv2a_GmCgKfI7cZhgSIR2zOg'
gc = gspread.service_account(filename=r'GoogleAPICredentials.json') # <--Key
sh = gc.open_by_key(sheet_id) # <-- sheet id
worksheet = sh.worksheet('test2') #->gg sheet name

# APPEND DATA TO SHEET
your_dataframe = final
set_with_dataframe(worksheet, your_dataframe) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET